<a href="https://colab.research.google.com/github/bala1802/ERA_Session17/blob/main/S17_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Session-17

/content/drive/MyDrive/Session-17


In [3]:
!pip install torchinfo --q

In [4]:
from torch.utils.data import Dataset
import torch.nn.functional as F
import transformer
import re
from os.path import exists
from collections import Counter
import random
import torch
from torchinfo import summary
from tqdm.auto import tqdm
import numpy as np

In [5]:
n_vocab = 40000
seq_len = 20
p_random_mask = 0.15
batch_size = 16
n_iterations = 20
epochs = 2
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
data_pth = 'data/training.txt'
sentences = open(data_pth).read().lower().split('\n')
special_chars = '?;.:/*!+-()[]{}"\'&'
sentences = [re.sub(f'[{re.escape(special_chars)}]','\g<0> ',s).split(' ') for s in sentences]
sentences = [[w for w in s if len(w)] for s in sentences]

In [7]:
# create vocab
vocab_pth = 'sentences_data/vocab.txt'
if not exists(vocab_pth):
  words = [w for s in sentences for w in s]
  vocab = Counter(words).most_common(n_vocab)
  vocab = [w[0] for w in vocab]
else:
  vocab = open(vocab_pth).read().split('\n')

In [8]:
class SentenceDataset(Dataset):
  def __init__(self,sentences,vocab,seq_len):
    dataset = self
    dataset.sentences = sentences
    dataset.vocab = vocab + ['<ignore>','<oov>','<mask>']
    dataset.vocab = {e:i for i,e in enumerate(dataset.vocab)}
    dataset.rvocab = {v:k for k,v in dataset.vocab.items()}

    dataset.seq_len = seq_len

    dataset.IGNORE_IDX = dataset.vocab['<ignore>']
    dataset.OUT_OF_VOCAB_IDX = dataset.vocab['<oov>']
    dataset.MASK_IDX = dataset.vocab['<mask>']

  def __getitem__(self,index,p_random_mask=0.15):
    dataset = self

    s = []
    while len(s) < dataset.seq_len:
      s.extend(dataset.get_sentence_idx(index % len(dataset)))
      index += 1

    s = s[:dataset.seq_len]
    [s.append(dataset.IGNORE_IDX) for i in range(dataset.seq_len - len(s))]
    s = [(dataset.MASK_IDX,w) if random.random() < p_random_mask else (w,dataset.IGNORE_IDX) for w in s]

    return {'input': torch.Tensor([w[0] for w in s]).long(),
            'target':torch.Tensor([w[1] for w in s]).long()}

  def __len__(self):
    return len(self.sentences)

  def get_sentence_idx(self,index):
    dataset = self
    s = dataset.sentences[index]
    s = [dataset.vocab[w] if w in dataset.vocab else dataset.OUT_OF_VOCAB_IDX for w in s]
    return s

In [9]:
# create dataset and train/test data
print('creating dataset...')
dataset = SentenceDataset(sentences, vocab, seq_len)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
print(f"size of train {train_size} and size of test {test_size}")

#dataloader = torch.utils.data.DataLoader(dataset,shuffle=True, drop_last=True, pin_memory=False, batch_size=batch_size)
train_data, test_data = torch.utils.data.random_split(dataset, [train_size, test_size])

train_dataloader = torch.utils.data.DataLoader(train_data,batch_size = batch_size,
                              shuffle=True,num_workers=2,
                              pin_memory=True)
test_dataloader = torch.utils.data.DataLoader(test_data,batch_size = batch_size,
                              shuffle=False,num_workers=2,
                              pin_memory=True)


# sample sentence
batch_output = next(iter(train_dataloader))
input_sentence, input_label = batch_output['input'][0], batch_output['target'][0]
print(input_sentence, input_label)

creating dataset...
size of train 99565 and size of test 24892
tensor([    1,     2,     6,   962,    59, 29198,     3,    31,   481,     6,
          962,    52,    49, 10273,    30,     9,   184,   777,    25,   327]) tensor([40000, 40000, 40000, 40000, 40000, 40000, 40000, 40000, 40000, 40000,
        40000, 40000, 40000, 40000, 40000, 40000, 40000, 40000, 40000, 40000])


In [10]:
bert_model = transformer.Bert(n_embeddings = len(dataset.vocab))
bert_model.to(device)
optimizer = torch.optim.Adam(params = bert_model.parameters(),lr=1e-4,betas=(0.9,0.999),
                             weight_decay=1e-4)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=dataset.IGNORE_IDX)
summary(model= bert_model, input_size=(32,20), dtypes = [torch.int32],col_names=["input_size","output_size","num_params","trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
Bert (Bert)                                                  [32, 20]             [32, 20, 40003]      2,560                True
├─Embedding (embeddings)                                     [32, 20]             [32, 20, 128]        5,120,384            True
├─Dropout (embedding_dropout)                                [32, 20, 128]        [32, 20, 128]        --                   --
├─Sequential (transformer_encoder)                           [32, 20, 128]        [32, 20, 128]        --                   True
│    └─TransformerEncoderBlock (0)                           [32, 20, 128]        [32, 20, 128]        --                   True
│    │    └─MultiHeadAttentionBlock (msa_block)              [32, 20, 128]        [32, 20, 128]        66,304               True
│    │    └─MLPBlock (mlp_block)                             [32, 20, 128]        [32, 20, 128

In [11]:
for epoch in tqdm(range(epochs)):

  # train model
  bert_model.train()
  train_loss = 0

  for batch_idx, batch_data in enumerate(train_dataloader):

    #infer
    masked_input  = batch_data['input']
    masked_target = batch_data['target']

    masked_input  = masked_input.to(device)
    masked_target = masked_target.to(device)
    output_pred = bert_model(masked_input)

    # compute the cross entropy loss
    output_v = output_pred.view(-1,output_pred.shape[-1])
    target_v = masked_target.view(-1,1).squeeze()
    loss = loss_fn(output_v, target_v)

    train_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

  train_loss = train_loss / len(train_dataloader)

  # test model
  bert_model.eval()
  test_loss= 0

  with torch.inference_mode():
    for batch_idx, batch_data in enumerate(test_dataloader):
      #infer
      masked_input  = batch_data['input']
      masked_target = batch_data['target']

      masked_input  = masked_input.to(device)
      masked_target = masked_target.to(device)
      output_pred = bert_model(masked_input)

      #compute the cross entropy loss
      output_v = output_pred.view(-1,output_pred.shape[-1])
      target_v = masked_target.view(-1,1).squeeze()
      loss = loss_fn(output_v, target_v)

      test_loss += loss.item()

    test_loss = test_loss / len(test_dataloader)


  print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
           f"test_loss: {test_loss:.4f} | "
          f"| Δw:, {round(bert_model.embeddings.weight.grad.abs().sum().item(),3)}"

  )

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 7.4742 | test_loss: 7.2325 | | Δw:, 4.327
Epoch: 2 | train_loss: 7.1677 | test_loss: 7.0806 | | Δw:, 8.989


In [12]:
def get_batch(loader,loader_iter):
  try:
        batch = next(loader_iter)
  except StopIteration:
      loader_iter = iter(loader)
      batch = next(loader_iter)
  return batch, loader_iter

In [13]:
batch_iter = iter(train_dataloader)
bert_model.train()
print_each = 10

for it in range(n_iterations):
  #get batch
  batch_data, batch_iter = get_batch(train_dataloader, batch_iter)

  masked_input  = batch_data['input']
  masked_target = batch_data['target']

  masked_input  = masked_input.to(device)
  masked_target = masked_target.to(device)
  output_pred   = bert_model(masked_input)

  #compute the cross entropy loss
  output_v = output_pred.view(-1,output_pred.shape[-1])
  target_v = masked_target.view(-1,1).squeeze()
  loss = loss_fn(output_v, target_v)

  train_loss += loss.item()

  optimizer.zero_grad()
  loss.backward()

  optimizer.step()

  #print step
  if it % print_each == 0:
      print('it:', it,
            ' | loss', np.round(loss.item(),2),
            ' | Δw:', round(bert_model.embeddings.weight.grad.abs().sum().item(),3))

it: 0  | loss 7.19  | Δw: 8.666
it: 10  | loss 6.81  | Δw: 9.914
